## 5 Testing attack

Брутфорс по самым топовым hero и poor пользователям (ROC-AUC Diff 0,250559 для топ 10 hero_users_top и poor_users_top и рельсой 1-15 по первым и последним 5 транзакциям - `5last_brute_2.py`)

In [1]:
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
# from tqdm import tqdm
import random
from model import predict # Функция, позволяет получить предсказание нейронки.
from check_budget import check_budget # функция проверки бюджета. Проверяйте допустимость решения до сабмита
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

bins_path = "nn_bins.pickle" # путь до файла с бинами после тренировки модели (nn_bins.pickle)
model_path = "nn_weights.ckpt" # путь до файла с весами нейронной сети (nn_weights.ckpt)
quantiles_path = "quantiles.json" # путь до файла с квантилями для таргета (quantiles.pickle)

transactions_path = "../sample_submission.csv"    # путь до файла с транзакциями, которые атакуются

train_data_path = "../transactions_finetune.csv" # X - train data
train_target_path = "../target_finetune.csv" # y - true target

In [3]:
seed = 26041999
BUDGET = 10 # разрешенное количество изменений транзакций для каждого пользователя - N транзакций
threshold = pd.read_csv(train_target_path).target.mean() # примерный threshold трейна

In [15]:
random.seed(20230206)

# threshold = pd.read_csv("../data/target_finetune.csv").target.mean()  # вероятность, по которой мы считаем таргет
hero_user = 829187
poor_user = 488752

df_transactions = pd.read_csv(
    transactions_path,
    parse_dates=["transaction_dttm"],
    dtype={"user_id": int, "mcc_code": int, "currency_rk": int, "transaction_amt": float},
)

bins_path = "nn_bins.pickle"
model_path = "nn_weights.ckpt"
target = predict(transactions_path, bins_path, model_path)


one_idx = target.index[target.target > threshold]  # Эти пользователи похожи на Героя
zero_idx = target.index[target.target <= threshold] # А эти на Неудачника

users = target.user_id.values

one_users = users[one_idx]
zero_users = users[zero_idx]

for user in tqdm(users):
    if user in one_users:
        copy_from = poor_user # похожим на Героя скопируем 10 последних транзакций Неудачника
    else:
        copy_from = hero_user # А похожим на Неудачника наоборот

    idx_to = df_transactions.index[df_transactions.user_id == user][-BUDGET:]
    idx_from = df_transactions.index[df_transactions.user_id == copy_from][-BUDGET:]
    sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
    sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
    sign_mask = (sign_to == sign_from)
    df_transactions.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
    df_transactions.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values
attacked_path = 'test.csv'
df_transactions.to_csv(attacked_path, index=False)

  0%|          | 0/4200 [00:00<?, ?it/s]

In [46]:
random.seed(20230206)
###############################################
# testing df_train last 10
##############################################
# hero_user = 829187
# poor_user = 488752
hero_user = 391362
poor_user = 233068

# threshold = pd.read_csv("../data/target_finetune.csv").target.mean()  # вероятность, по которой мы считаем таргет

df_transactions = pd.read_csv(
    transactions_path,
    parse_dates=["transaction_dttm"],
    dtype={"user_id": int, "mcc_code": int, "currency_rk": int, "transaction_amt": float},
)

bins_path = "nn_bins.pickle"
model_path = "nn_weights.ckpt"
target = predict(transactions_path, bins_path, model_path)

one_idx = target.index[target.target > threshold]  # Эти пользователи похожи на Героя
zero_idx = target.index[target.target <= threshold] # А эти на Неудачника

users = target.user_id.values

one_users = users[one_idx] # defolt - 1
zero_users = users[zero_idx] # norm users - 0

for user in tqdm(users):
    if user in one_users:
        copy_from = poor_user
    else:
        copy_from = hero_user

#     idx_to_first = df_transactions.index[df_transactions.user_id == user][:5] # айдишники последних 10 транзакций юзера
#     idx_from_first = df_transactions.index[df_transactions.user_id == copy_from][:5] # айдишники последних 10 транзакций воннаби

#     idx_to_last = df_transactions.index[df_transactions.user_id == user][-5:] # айдишники последних 10 транзакций юзера
#     idx_from_last = df_transactions.index[df_transactions.user_id == copy_from][-5:] # айдишники последних 10 транзакций воннаби

    idx_to_first = df_transactions.index[df_transactions.user_id == user][:10:2] # айдишники последних 10 транзакций юзера
    idx_from_first = df_transactions.index[df_transactions.user_id == copy_from][:10:2] # айдишники последних 10 транзакций воннаби

    idx_to_last = df_transactions.index[df_transactions.user_id == user][-10::2] # айдишники последних 10 транзакций юзера
    idx_from_last = df_transactions.index[df_transactions.user_id == copy_from][-10::2] # айдишники последних 10 транзакций воннаби

    idx_to = idx_to_first.append(idx_to_last)
    idx_from = idx_from_first.append(idx_from_last)

    sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
    sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
    sign_mask = (sign_to == sign_from)

    df_transactions.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
    df_transactions.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values

attacked_path = 'test.csv'
df_transactions.to_csv(attacked_path, index=False)

Global seed set to 26041999


  0%|          | 0/4200 [00:00<?, ?it/s]

In [3]:
pred = predict('../sample_submission.csv', bins_path, model_path, random_seed=26041999)
pred['target_true'] = pred.target.apply(lambda x: 0 if x <= threshold else 1)

Global seed set to 26041999


In [48]:
pred_attacked = predict(attacked_path, bins_path, model_path, random_seed=26041999)
pred_attacked

Global seed set to 26041999


,user_id,target
0,66,0.067373
1,213,0.038757
2,345,0.127810
3,569,0.029676
4,573,0.036346
...,...,...
4195,868183,0.039675
4196,868407,0.132915
4197,868573,0.112722
4198,868694,0.035121


In [14]:
# [:BUDGET]
# hero_user = 829187
# poor_user = 488752
roc_auc_score(pred['target_true'], pred_attacked['target'])

0.777240397236187

In [18]:
# [-BUDGET:]
# hero_user = 829187
# poor_user = 488752
roc_auc_score(pred['target_true'], pred_attacked['target'])

0.7363024145468998

In [23]:
# [:5] [-5:]
# hero_user = 829187
# poor_user = 488752
roc_auc_score(pred['target_true'], pred_attacked['target'])

0.6415783967861705

In [27]:
# [:5] [-5:]
# hero_user = 531067
# poor_user = 830119
roc_auc_score(pred['target_true'], pred_attacked['target'])

0.8411920609924154

In [6]:
# random state [:5] [-5:]
# hero_user = 829187
# poor_user = 488752
roc_auc_score(pred['target_true'], pred_attacked['target'])

0.5881111081374695

In [18]:
# random state [:5] [:10:2] and [-5:] [-10::2]
# hero_user = 829187
# poor_user = 488752
roc_auc_score(pred['target_true'], pred_attacked['target'])

0.4689471276604181

In [49]:
# random state ...
# hero_user = 829187
# poor_user = 488752
roc_auc_score(pred['target_true'], pred_attacked['target'])

0.8689179116313002

In [7]:
pred_attacked = predict('last_brute.csv', bins_path, model_path, random_seed=26041999)

Global seed set to 26041999


In [8]:
# 5last_brute.py
roc_auc_score(pred['target_true'], pred_attacked['target'])

0.2777801319107432

In [6]:
check_budget(transactions_path, 'last_brute.csv', quantiles_path)

100%|█████████████████████████████████████████████████████████████████████| 1260000/1260000 [00:34<00:00, 36971.69it/s]


True

#### USERS BRUTE

In [38]:
pred = predict('../sample_submission.csv', bins_path, model_path, random_seed=26041999)
pred['target_true'] = pred.target.apply(lambda x: 0 if x <= threshold else 1)

Global seed set to 26041999


In [ ]:
# hero_user = 391362
# poor_user = 233068

In [75]:
hero_users_top = pred.loc[(pred.target_true == 1)].sort_values('target')[-100:].user_id.values
poor_users_top = pred.loc[(pred.target_true == 0)].sort_values('target')[:100].user_id.values

In [77]:
import pytorch_lightning as pl
import torch
import pickle
from model import process_for_nn, TransactionsDataset, get_dataloader, device, TransactionsRnn

pl.seed_everything(seed)
model = TransactionsRnn()
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()
with open(bins_path, "rb") as f:
    bins = pickle.load(f)
# features = bins.pop("features")


def create_dl(df, bins_path):
    df_transactions = df.dropna().assign(
            hour=lambda x: x.transaction_dttm.dt.hour,
            day=lambda x: x.transaction_dttm.dt.dayofweek,
            month=lambda x: x.transaction_dttm.dt.month,
            number_day=lambda x: x.transaction_dttm.dt.day,
        )

    with open(bins_path, "rb") as f:
        bins = pickle.load(f)
    features = bins.pop("features")
    # features = bins["features"]  ########################## мб можно оптимизировать
    df = process_for_nn(df_transactions, features, bins)
    dataset = TransactionsDataset(df)
    dataloader = get_dataloader(dataset, device, is_validation=True)
    return dataloader


def predictor(dataloader, model):
    preds = []
    users = []
    for data, target in dataloader:
        y_pred = model(data)
        preds.append(y_pred.detach().cpu().numpy())
        users.append(target.detach().cpu().numpy())
    preds = np.concatenate(preds)
    users = np.concatenate(users)
    return pd.DataFrame({"user_id": users, "target": preds[:, 1]})


def check_target_attack(user, target, attacked_target):
    if target.target_true.loc[target.user_id == user].values[0]:
        return target.target.loc[target.user_id == user].values[0] - attacked_target.target.loc[attacked_target.user_id == user].values[0]
    else:
        return attacked_target.target.loc[attacked_target.user_id == user].values[0] - target.target.loc[target.user_id == user].values[0]

Global seed set to 26041999


In [82]:
random.seed(20230206)

df_transactions = pd.read_csv(
    transactions_path,
    parse_dates=["transaction_dttm"],
    dtype={"user_id": int, "mcc_code": int, "currency_rk": int, "transaction_amt": float},
)

df_transactions_attacked = df_transactions.copy()

bins_path = "nn_bins.pickle"
model_path = "nn_weights.ckpt"
target = predict(transactions_path, bins_path, model_path)
target['target_true'] = target.target.apply(lambda x: 0 if x <= threshold else 1)

one_idx = target.index[target.target > threshold]  # Эти пользователи похожи на Героя
zero_idx = target.index[target.target <= threshold] # А эти на Неудачника

users = target.user_id.values

one_users = users[one_idx] # defolt - 1
zero_users = users[zero_idx] # norm users - 0

for user in tqdm(users[:30], desc='Total'):
    df_transactions_user = df_transactions.loc[df_transactions.user_id == user].copy()
    user_score = 0
    best_copy_from = 829187
    if user in one_users:
        for copy_from in tqdm(poor_users_top, leave=False, desc='poor_users_top'):
            idx_to_first = df_transactions_user.index[df_transactions_user.user_id == user][:5] # айдишники последних 10 транзакций юзера
            idx_from_first = df_transactions.index[df_transactions.user_id == copy_from][:10:2] # айдишники последних 10 транзакций воннаби

            idx_to_last = df_transactions_user.index[df_transactions_user.user_id == user][-5:] # айдишники последних 10 транзакций юзера
            idx_from_last = df_transactions.index[df_transactions.user_id == copy_from][-10::2] # айдишники последних 10 транзакций воннаби

            idx_to = idx_to_first.append(idx_to_last)
            idx_from = idx_from_first.append(idx_from_last)

            sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
            sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
            sign_mask = (sign_to == sign_from)

            df_transactions_user.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
            df_transactions_user.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values

            dataloader = create_dl(df_transactions_user, bins_path)

            attacked_target = predictor(dataloader, model)

            current_user_score = check_target_attack(user, target, attacked_target)

            if current_user_score >= user_score:
                user_score = current_user_score
                best_copy_from = copy_from

    else:
        for copy_from in tqdm(hero_users_top, leave=False, desc='hero_users_top'):
            idx_to_first = df_transactions_user.index[df_transactions_user.user_id == user][:5] # айдишники последних 10 транзакций юзера
            idx_from_first = df_transactions.index[df_transactions.user_id == copy_from][:10:2] # айдишники последних 10 транзакций воннаби

            idx_to_last = df_transactions_user.index[df_transactions_user.user_id == user][-5:] # айдишники последних 10 транзакций юзера
            idx_from_last = df_transactions.index[df_transactions.user_id == copy_from][-10::2] # айдишники последних 10 транзакций воннаби

            idx_to = idx_to_first.append(idx_to_last)
            idx_from = idx_from_first.append(idx_from_last)

            sign_to = np.sign(df_transactions_user.loc[idx_to, "transaction_amt"].values)
            sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
            sign_mask = (sign_to == sign_from)

            df_transactions_user.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
            df_transactions_user.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values

            dataloader = create_dl(df_transactions_user, bins_path)

            attacked_target = predictor(dataloader, model)

            current_user_score = check_target_attack(user, target, attacked_target)

            if current_user_score >= user_score:
                user_score = current_user_score
                best_copy_from = copy_from

    idx_to_first = df_transactions.index[df_transactions.user_id == user][:5] # айдишники последних 10 транзакций юзера
    idx_from_first = df_transactions.index[df_transactions.user_id == best_copy_from][:10:2] # айдишники последних 10 транзакций воннаби

    idx_to_last = df_transactions.index[df_transactions.user_id == user][-5:] # айдишники последних 10 транзакций юзера
    idx_from_last = df_transactions.index[df_transactions.user_id == best_copy_from][-10::2] # айдишники последних 10 транзакций воннаби

    idx_to = idx_to_first.append(idx_to_last)
    idx_from = idx_from_first.append(idx_from_last)

    sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
    sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
    sign_mask = (sign_to == sign_from)

    df_transactions_attacked.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
    df_transactions_attacked.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values


attacked_path = 'last_brute_check.csv'
df_transactions_attacked.to_csv(attacked_path, index=False)

Global seed set to 26041999


Total:   0%|          | 0/30 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/100 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/100 [00:00<?, ?it/s]

poor_users_top:   0%|          | 0/100 [00:00<?, ?it/s]

...

hero_users_top:   0%|          | 0/100 [00:00<?, ?it/s]

poor_users_top:   0%|          | 0/100 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/100 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/100 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/100 [00:00<?, ?it/s]

In [62]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import random
from model import predict # Функция, позволяет получить предсказание нейронки.

import pytorch_lightning as pl
import torch
import pickle
from model import process_for_nn, TransactionsDataset, get_dataloader, device, TransactionsRnn

bins_path = "nn_bins.pickle" # путь до файла с бинами после тренировки модели (nn_bins.pickle)
model_path = "nn_weights.ckpt" # путь до файла с весами нейронной сети (nn_weights.ckpt)
quantiles_path = "quantiles.json" # путь до файла с квантилями для таргета (quantiles.pickle)

transactions_path = "../sample_submission.csv"    # путь до файла с транзакциями, которые атакуются

train_data_path = "../transactions_finetune.csv" # X - train data
train_target_path = "../target_finetune.csv" # y - true target

threshold = pd.read_csv(train_target_path).target.mean() # примерный threshold трейна


def create_dl(df, bins_path):
    df_transactions = df.dropna().assign(
            hour=lambda x: x.transaction_dttm.dt.hour,
            day=lambda x: x.transaction_dttm.dt.dayofweek,
            month=lambda x: x.transaction_dttm.dt.month,
            number_day=lambda x: x.transaction_dttm.dt.day,
        )

    with open(bins_path, "rb") as f:
        bins = pickle.load(f)
    features = bins.pop("features")
    # features = bins["features"]  ########################## мб можно оптимизировать
    df = process_for_nn(df_transactions, features, bins)
    dataset = TransactionsDataset(df)
    dataloader = get_dataloader(dataset, device, is_validation=True)
    return dataloader


def predictor(dataloader, model):
    preds = []
    users = []
    for data, target in dataloader:
        y_pred = model(data)
        preds.append(y_pred.detach().cpu().numpy())
        users.append(target.detach().cpu().numpy())
    preds = np.concatenate(preds)
    users = np.concatenate(users)
    return pd.DataFrame({"user_id": users, "target": preds[:, 1]})


def check_target_attack(user, target, attacked_target):
    if target.target_true.loc[target.user_id == user].values[0]:
        return target.target.loc[target.user_id == user].values[0] - attacked_target.target.loc[attacked_target.user_id == user].values[0]
    else:
        return attacked_target.target.loc[attacked_target.user_id == user].values[0] - target.target.loc[target.user_id == user].values[0]

pred = predict('../sample_submission.csv', bins_path, model_path, random_seed=26041999)
pred['target_true'] = pred.target.apply(lambda x: 0 if x <= threshold else 1)

hero_users_top = pred.loc[(pred.target_true == 1)].sort_values('target')[-80:].user_id.values
poor_users_top = pred.loc[(pred.target_true == 0)].sort_values('target')[:80].user_id.values

pl.seed_everything(26041999)
model = TransactionsRnn()
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()
with open(bins_path, "rb") as f:
    bins = pickle.load(f)

random.seed(20230206)
###############################################


df_transactions = pd.read_csv(
    transactions_path,
    parse_dates=["transaction_dttm"],
    dtype={"user_id": int, "mcc_code": int, "currency_rk": int, "transaction_amt": float},
)

df_transactions_attacked = df_transactions.copy()

bins_path = "nn_bins.pickle"
model_path = "nn_weights.ckpt"
target = predict(transactions_path, bins_path, model_path)
target['target_true'] = target.target.apply(lambda x: 0 if x <= threshold else 1)

one_idx = target.index[target.target > threshold]  # Эти пользователи похожи на Героя
zero_idx = target.index[target.target <= threshold] # А эти на Неудачника

users = target.user_id.values

one_users = users[one_idx] # defolt - 1
zero_users = users[zero_idx] # norm users - 0

for user in tqdm(users[:10], desc='Total'):
    df_transactions_user = df_transactions.loc[df_transactions.user_id == user].copy()
    user_score = 0
    best_copy_from = 829187
    best_i = 1
    if user in one_users:
        for copy_from in tqdm(poor_users_top, leave=False, desc='poor_users_top'):
            for i in range(1, 13):
                idx_to_first = df_transactions_user.index[df_transactions_user.user_id == user][:5] # айдишники первых 5 транзакций юзера
                idx_from_first = df_transactions.index[df_transactions.user_id == copy_from][i:5+i] # айдишники первых 5 транзакций воннаби

                idx_to_last = df_transactions_user.index[df_transactions_user.user_id == user][-5:] # айдишники последних 5 транзакций юзера
                idx_from_last = df_transactions.index[df_transactions.user_id == copy_from][-5-i:-i] # айдишники последних 5 транзакций воннаби

                idx_to = idx_to_first.append(idx_to_last)
                idx_from = idx_from_first.append(idx_from_last)

                sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
                sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
                sign_mask = (sign_to == sign_from)

                df_transactions_user.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
                df_transactions_user.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values

                dataloader = create_dl(df_transactions_user, bins_path)

                attacked_target = predictor(dataloader, model)

                current_user_score = check_target_attack(user, target, attacked_target)

                if current_user_score >= user_score:
                    user_score = current_user_score
                    best_copy_from = copy_from
                    best_i = i

    else:
        for copy_from in tqdm(hero_users_top, leave=False, desc='hero_users_top'):
            for i in range(1, 13):
                idx_to_first = df_transactions_user.index[df_transactions_user.user_id == user][:5] # айдишники первых 5 транзакций юзера
                idx_from_first = df_transactions.index[df_transactions.user_id == copy_from][i:5+i] # айдишники первых 5 транзакций воннаби

                idx_to_last = df_transactions_user.index[df_transactions_user.user_id == user][-5:] # айдишники последних 5 транзакций юзера
                idx_from_last = df_transactions.index[df_transactions.user_id == copy_from][-5-i:-i] # айдишники последних 5 транзакций воннаби

                idx_to = idx_to_first.append(idx_to_last)
                idx_from = idx_from_first.append(idx_from_last)

                sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
                sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
                sign_mask = (sign_to == sign_from)

                df_transactions_user.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
                df_transactions_user.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values

                dataloader = create_dl(df_transactions_user, bins_path)

                attacked_target = predictor(dataloader, model)

                current_user_score = check_target_attack(user, target, attacked_target)

                if current_user_score >= user_score:
                    user_score = current_user_score
                    best_copy_from = copy_from
                    best_i = i

    idx_to_first = df_transactions.index[df_transactions.user_id == user][:5]
    idx_from_first = df_transactions.index[df_transactions.user_id == best_copy_from][best_i:5+best_i]

    idx_to_last = df_transactions.index[df_transactions.user_id == user][-5:]
    idx_from_last = df_transactions.index[df_transactions.user_id == best_copy_from][-5-best_i:-best_i]

    idx_to = idx_to_first.append(idx_to_last)
    idx_from = idx_from_first.append(idx_from_last)

    sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
    sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
    sign_mask = (sign_to == sign_from)

    df_transactions_attacked.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
    df_transactions_attacked.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values


attacked_path = 'last_brute_split_1_test.csv'
df_transactions_attacked.to_csv(attacked_path, index=False)

Global seed set to 26041999
Global seed set to 26041999
Global seed set to 26041999


Total:   0%|          | 0/10 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/80 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/80 [00:00<?, ?it/s]

poor_users_top:   0%|          | 0/80 [00:00<?, ?it/s]

...

hero_users_top:   0%|          | 0/80 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/80 [00:00<?, ?it/s]

poor_users_top:   0%|          | 0/80 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/80 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/80 [00:00<?, ?it/s]

In [63]:
roc_auc_score(pred[:10]['target_true'], pred_attacked[:10]['target'])

0.1428571428571429

In [64]:
pred_attacked2 = predict('last_brute_split_1_test.csv', bins_path, model_path, random_seed=26041999)

Global seed set to 26041999


In [65]:
roc_auc_score(pred[:10]['target_true'], pred_attacked2[:10]['target'])

0.09523809523809526

In [67]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import random
from model import predict # Функция, позволяет получить предсказание нейронки.

import pytorch_lightning as pl
import torch
import pickle
from model import process_for_nn, TransactionsDataset, get_dataloader, device, TransactionsRnn

bins_path = "nn_bins.pickle" # путь до файла с бинами после тренировки модели (nn_bins.pickle)
model_path = "nn_weights.ckpt" # путь до файла с весами нейронной сети (nn_weights.ckpt)
quantiles_path = "quantiles.json" # путь до файла с квантилями для таргета (quantiles.pickle)

transactions_path = "../sample_submission.csv"    # путь до файла с транзакциями, которые атакуются

train_data_path = "../transactions_finetune.csv" # X - train data
train_target_path = "../target_finetune.csv" # y - true target

threshold = pd.read_csv(train_target_path).target.mean() # примерный threshold трейна


def create_dl(df, bins_path):
    df_transactions = df.dropna().assign(
            hour=lambda x: x.transaction_dttm.dt.hour,
            day=lambda x: x.transaction_dttm.dt.dayofweek,
            month=lambda x: x.transaction_dttm.dt.month,
            number_day=lambda x: x.transaction_dttm.dt.day,
        )

    with open(bins_path, "rb") as f:
        bins = pickle.load(f)
    features = bins.pop("features")
    # features = bins["features"]  ########################## мб можно оптимизировать
    df = process_for_nn(df_transactions, features, bins)
    dataset = TransactionsDataset(df)
    dataloader = get_dataloader(dataset, device, is_validation=True)
    return dataloader


def predictor(dataloader, model):
    preds = []
    users = []
    for data, target in dataloader:
        y_pred = model(data)
        preds.append(y_pred.detach().cpu().numpy())
        users.append(target.detach().cpu().numpy())
    preds = np.concatenate(preds)
    users = np.concatenate(users)
    return pd.DataFrame({"user_id": users, "target": preds[:, 1]})


def check_target_attack(user, target, attacked_target):
    if target.target_true.loc[target.user_id == user].values[0]:
        return target.target.loc[target.user_id == user].values[0] - attacked_target.target.loc[attacked_target.user_id == user].values[0]
    else:
        return attacked_target.target.loc[attacked_target.user_id == user].values[0] - target.target.loc[target.user_id == user].values[0]

pred = predict('../sample_submission.csv', bins_path, model_path, random_seed=26041999)
pred['target_true'] = pred.target.apply(lambda x: 0 if x <= threshold else 1)

hero_users_top = pred.loc[(pred.target_true == 1)].sort_values('target')[-20:].user_id.values
poor_users_top = pred.loc[(pred.target_true == 0)].sort_values('target')[:20].user_id.values

pl.seed_everything(26041999)
model = TransactionsRnn()
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()
with open(bins_path, "rb") as f:
    bins = pickle.load(f)

random.seed(20230206)
###############################################
# testing df_train last 10
##############################################

# threshold = pd.read_csv("../data/target_finetune.csv").target.mean()  # вероятность, по которой мы считаем таргет

df_transactions = pd.read_csv(
    transactions_path,
    parse_dates=["transaction_dttm"],
    dtype={"user_id": int, "mcc_code": int, "currency_rk": int, "transaction_amt": float},
)

df_transactions_attacked = df_transactions.copy()

bins_path = "nn_bins.pickle"
model_path = "nn_weights.ckpt"
target = predict(transactions_path, bins_path, model_path)
target['target_true'] = target.target.apply(lambda x: 0 if x <= threshold else 1)

one_idx = target.index[target.target > threshold]  # Эти пользователи похожи на Героя
zero_idx = target.index[target.target <= threshold] # А эти на Неудачника

users = target.user_id.values

one_users = users[one_idx] # defolt - 1
zero_users = users[zero_idx] # norm users - 0

for user in tqdm(users[:10], desc='Total'):
    df_transactions_user = df_transactions.loc[df_transactions.user_id == user].copy()
    user_score = 0
    best_copy_from = 829187
    best_i = 1
    if user in one_users:
        for copy_from in tqdm(poor_users_top, leave=False, desc='poor_users_top'):
            for i in range(1, 20):
                idx_to_first = df_transactions_user.index[df_transactions_user.user_id == user][:5] # айдишники первых 5 транзакций юзера
                idx_from_first = df_transactions.index[df_transactions.user_id == copy_from][:5*i:i] # айдишники первых 5 транзакций воннаби

                idx_to_last = df_transactions_user.index[df_transactions_user.user_id == user][-5:] # айдишники последних 5 транзакций юзера
                idx_from_last = df_transactions.index[df_transactions.user_id == copy_from][-5*i::i] # айдишники последних 5 транзакций воннаби

                idx_to = idx_to_first.append(idx_to_last)
                idx_from = idx_from_first.append(idx_from_last)

                sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
                sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
                sign_mask = (sign_to == sign_from)

                df_transactions_user.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
                df_transactions_user.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values

                dataloader = create_dl(df_transactions_user, bins_path)

                attacked_target = predictor(dataloader, model)

                current_user_score = check_target_attack(user, target, attacked_target)

                if current_user_score >= user_score:
                    user_score = current_user_score
                    best_copy_from = copy_from
                    best_i = i

    else:
        for copy_from in tqdm(hero_users_top, leave=False, desc='hero_users_top'):
            for i in range(1, 20):
                idx_to_first = df_transactions_user.index[df_transactions_user.user_id == user][:5] # айдишники первых 5 транзакций юзера
                idx_from_first = df_transactions.index[df_transactions.user_id == copy_from][:5*i:i] # айдишники первых 5 транзакций воннаби

                idx_to_last = df_transactions_user.index[df_transactions_user.user_id == user][-5:] # айдишники последних 5 транзакций юзера
                idx_from_last = df_transactions.index[df_transactions.user_id == copy_from][-5*i::i] # айдишники последних 5 транзакций воннаби

                idx_to = idx_to_first.append(idx_to_last)
                idx_from = idx_from_first.append(idx_from_last)

                sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
                sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
                sign_mask = (sign_to == sign_from)

                df_transactions_user.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
                df_transactions_user.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values

                dataloader = create_dl(df_transactions_user, bins_path)

                attacked_target = predictor(dataloader, model)

                current_user_score = check_target_attack(user, target, attacked_target)

                if current_user_score >= user_score:
                    user_score = current_user_score
                    best_copy_from = copy_from
                    best_i = i

    idx_to_first = df_transactions.index[df_transactions.user_id == user][:5]
    idx_from_first = df_transactions.index[df_transactions.user_id == best_copy_from][:5*best_i:best_i]

    idx_to_last = df_transactions.index[df_transactions.user_id == user][-5:]
    idx_from_last = df_transactions.index[df_transactions.user_id == best_copy_from][-5*best_i::best_i]

    idx_to = idx_to_first.append(idx_to_last)
    idx_from = idx_from_first.append(idx_from_last)

    sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
    sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
    sign_mask = (sign_to == sign_from)

    df_transactions_attacked.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
    df_transactions_attacked.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values


attacked_path = 'last_brute_split_2_test.csv'
df_transactions_attacked.to_csv(attacked_path, index=False)

Global seed set to 26041999
Global seed set to 26041999
Global seed set to 26041999


Total:   0%|          | 0/10 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

poor_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

...

hero_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

poor_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

In [68]:
pred_attacked3 = predict('last_brute_split_2_test.csv', bins_path, model_path, random_seed=26041999)

Global seed set to 26041999


In [47]:
roc_auc_score(pred[:10]['target_true'], pred_attacked[:10]['target'])

0.1428571428571429

In [48]:
roc_auc_score(pred[:10]['target_true'], pred_attacked3[:10]['target'])

0.14285714285714285

In [56]:
roc_auc_score(pred[:10]['target_true'], pred_attacked3[:10]['target'])

0.04761904761904763

In [61]:
# :80 range 15
roc_auc_score(pred[:10]['target_true'], pred_attacked3[:10]['target'])

0.09523809523809523

In [69]:
roc_auc_score(pred[:10]['target_true'], pred_attacked3[:10]['target'])

0.04761904761904763

In [53]:
pred_attacked[:10]

,user_id,target
0,66,0.124032
1,213,0.075672
2,345,0.066365
3,569,0.017604
4,573,0.067958
5,824,0.011650
6,1840,0.099187
7,1949,0.019046
8,2135,0.087735
9,2204,0.123523


In [58]:
pred_attacked3[:10]

,user_id,target
0,66,0.118982
1,213,0.156076
2,345,0.057281
3,569,0.023313
4,573,0.072538
5,824,0.025457
6,1840,0.107366
7,1949,0.019383
8,2135,0.095316
9,2204,0.099436


In [4]:
pred = predict('../sample_submission.csv', bins_path, model_path, random_seed=26041999)
pred['target_true'] = pred.target.apply(lambda x: 0 if x <= threshold else 1)

Global seed set to 26041999


In [5]:
pred_attacked2 = predict('last_brute_split_2.csv', bins_path, model_path, random_seed=26041999)
pred_attacked3 = predict('last_brute_split_1.csv', bins_path, model_path, random_seed=26041999)

Global seed set to 26041999
Global seed set to 26041999


In [9]:
roc_auc_score(pred['target_true'], pred_attacked2['target'])

0.2218653852253624